In [1]:
# 1. Preprocessing: Binning + Normalization + Tests + Confusion Matrix

import math

# --- Binning ---
data = [12, 15, 14, 10, 19, 23, 18, 17, 22, 30, 28, 5, 7, 9]

# Equal-width binning
def equal_width(data, k):
    mn, mx = min(data), max(data)
    width = (mx - mn) / k
    bins = [[] for _ in range(k)]
    for x in data:
        idx = min(k-1, int((x - mn) / width))
        bins[idx].append(x)
    return bins

# Equal-frequency binning
def equal_freq(data, k):
    s = sorted(data)
    size = len(s) // k
    return [s[i:i+size] for i in range(0, len(s), size)]

print("Equal-width:", equal_width(data, 3))
print("Equal-frequency:", equal_freq(data, 3))

# --- Normalization ---
def minmax(data):
    mn, mx = min(data), max(data)
    return [(x-mn)/(mx-mn) for x in data]

def zscore(data):
    m = sum(data)/len(data)
    s = math.sqrt(sum((x-m)**2 for x in data)/len(data))
    return [(x-m)/s for x in data]

print("Min-Max:", minmax(data))
print("Z-score:", zscore(data))

# --- Confusion Matrix ---
y_true = ["Pass","Fail","Pass","Fail"]
y_pred = ["Pass","Pass","Pass","Fail"]

def confusion_matrix(y_true, y_pred):
    labels = list(set(y_true+y_pred))
    matrix = {lab:{l:0 for l in labels} for lab in labels}
    for t,p in zip(y_true,y_pred):
        matrix[t][p]+=1
    return matrix

print("Confusion Matrix:", confusion_matrix(y_true,y_pred))


Equal-width: [[12, 10, 5, 7, 9], [15, 14, 19, 18, 17], [23, 22, 30, 28]]
Equal-frequency: [[5, 7, 9, 10], [12, 14, 15, 17], [18, 19, 22, 23], [28, 30]]
Min-Max: [0.28, 0.4, 0.36, 0.2, 0.56, 0.72, 0.52, 0.48, 0.68, 1.0, 0.92, 0.0, 0.08, 0.16]
Z-score: [-0.5972074699774571, -0.18601544146838833, -0.3230794509714113, -0.871335488983503, 0.3622405965437034, 0.9104966345557952, 0.22517658704068047, 0.08811257753765753, 0.7734326250527722, 1.8699447010769557, 1.5958166820709099, -1.5566555364986177, -1.2825275174925719, -1.0083994984865259]
Confusion Matrix: {'Pass': {'Pass': 2, 'Fail': 0}, 'Fail': {'Pass': 1, 'Fail': 1}}


In [6]:
# PCA on Iris using sklearn's load_iris (simpler)
from sklearn.datasets import load_iris
import math

# load iris dataset
iris = load_iris()
X = iris.data  # 150 x 4
y = iris.target  # class labels

# --- center the data ---
n, d = len(X), len(X[0])
means = [sum(row[j] for row in X)/n for j in range(d)]
Xc = [[row[j]-means[j] for j in range(d)] for row in X]

# --- covariance matrix ---
C = [[0.0]*d for _ in range(d)]
for i in range(d):
    for j in range(d):
        C[i][j] = sum(Xc[k][i]*Xc[k][j] for k in range(n))/(n-1)

print("Covariance matrix:")
for row in C:
    print([round(float(v), 4) for v in row])


Covariance matrix:
[0.6857, -0.0424, 1.2743, 0.5163]
[-0.0424, 0.19, -0.3297, -0.1216]
[1.2743, -0.3297, 3.1163, 1.2956]
[0.5163, -0.1216, 1.2956, 0.581]


In [7]:
# 3. FIND-S Algorithm
import csv

def find_s(path):
    with open(path) as f:
        rdr = csv.reader(f)
        header = next(rdr)
        data = [row for row in rdr]
    h = ["ϕ"]*(len(header)-1)
    for row in data:
        if row[-1].lower()=="yes":
            for i in range(len(h)):
                if h[i]=="ϕ": h[i]=row[i]
                elif h[i]!=row[i]: h[i]="?"
    return h

# Example training.csv
# Outlook,Temp,Humidity,Wind,Play
# Sunny,Hot,High,Weak,No
# Overcast,Hot,High,Weak,Yes
print("Hypothesis:", find_s("training.csv"))


Hypothesis: ['?', '?', '?', '?']


In [9]:
# 4. Candidate-Elimination Algorithm
import csv

def candidate_elim(path):
    with open(path) as f:
        rdr = csv.reader(f)
        header = next(rdr)
        data = [row for row in rdr]

    n = len(header)-1
    S = [["ϕ"]*n]     # most specific
    G = [["?"]*n]     # most general

    for row in data:
        x, y = row[:n], row[-1].lower()
        if y=="yes":
            # generalize S
            for s in S:
                for i in range(n):
                    if s[i]=="ϕ": s[i]=x[i]
                    elif s[i]!=x[i]: s[i]="?"
            G = [g for g in G if all(g[i]=="?" or g[i]==x[i] for i in range(n))]
        else:
            # specialize G
            newG=[]
            for g in G:
                for i in range(n):
                    if g[i]=="?":
                        for val in set(r[i] for r in data):
                            if val!=x[i]:
                                h=g[:]; h[i]=val; newG.append(h)
            G=newG
    return S,G

print("S,G:", candidate_elim("training.csv"))


S,G: ([['Sunny', 'Hot', '?', '?']], [['?', '?', '?', '?']])
